# Simulation Evaluation
In this notebook you are going to control both the SDV and other agents using a CNN-based policy in a closed-loop fashion.

**Note: to learn more about closed-loop evaluation refer to our [planning notebook](../planning/closed_loop_test.ipynb).**

**Note: this notebook assumes you've already run the [training notebook](./train.ipynb) and stored your model successfully.**

## What is ML Simulation?
TODO WRITE

## What can we use ML simulation for?
Simulating other agents is crucial to remove false positive interventions which occur because agents are log-replayed.
As an example, imagine if our SDV is slower compared to the data log. In this situation the car behind will crash into us
if it's just replaying the log. Differently, if also that agent is equipped with a ML policy it will be able to react and slow 
down before colliding with the SDV.

TODO PICTURE

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset, filter_agents_by_frames
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points, yaw_as_rotation33
from l5kit.visualization import TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, draw_trajectory

from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator
from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
from l5kit.tests.simulation.unroll_test import MockModel
import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./config.yaml")

## Load the model

In [ ]:
model_path = "/tmp/simulation_model.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path).to(device)
model = model.eval()
torch.set_grad_enabled(False)

## Load the evaluation data
Differently from the open loop evaluation, this setting is intrinsically sequential. As such, we won't be using any of PyTorch's parallelisation functionalities.

In [ ]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
print(eval_dataset)

## Define our evaluation protocol

In [ ]:
metrics = [DisplacementErrorL2Metric(),
           DistanceToRefTrajectoryMetric(),
           CollisionFrontMetric(),
           CollisionRearMetric(),
           CollisionSideMetric()]

validators = [RangeValidator("displacement_error_l2_validator", DisplacementErrorL2Metric, max_value=30),
              RangeValidator("distance_ref_trajectory_validator", DistanceToRefTrajectoryMetric, max_value=4),
              RangeValidator("collision_front_validator", CollisionFrontMetric, max_value=0),
              RangeValidator("collision_rear_validator", CollisionRearMetric, max_value=0),
              RangeValidator("collision_side_validator", CollisionSideMetric, max_value=0),
              ]

intervention_validators = ["displacement_error_l2_validator",
                           "distance_ref_trajectory_validator",
                           "collision_front_validator",
                           "collision_rear_validator",
                           "collision_side_validator"]

cle_evaluator = ClosedLoopEvaluator(EvaluationPlan(metrics=metrics,
                                    validators=validators,
                                    composite_metrics=[],
                                    intervention_validators=intervention_validators))
num_scenes_to_unroll = 2

# Measuring agents realism
TODO

In [ ]:
sim_cfg = SimulationConfig(use_ego_gt=True, use_agents_gt=False, disable_new_agents=True,
                           distance_th_far=500, distance_th_close=50, num_simulation_steps=20,
                           start_frame_index=0, show_info=True)

sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, torch.device("cpu"), model_agents=model)

sim_out = sim_loop.unroll(list(range(num_scenes_to_unroll)))

# Quantitative evaluation

## Plotting errors from the closed-loop

TODO maybe

# Qualitative evaluation

TODO

In [ ]:
# TODO when we have visualisation

# Measuring ego metrics with simulated agents
TODO

In [ ]:
sim_cfg = SimulationConfig(use_ego_gt=False, use_agents_gt=False, disable_new_agents=False,
                           distance_th_far=30, distance_th_close=15, num_simulation_steps=20,
                           start_frame_index=0, show_info=True)

sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, torch.device("cpu"), model_ego=model, model_agents=model)

sim_out = sim_loop.unroll(list(range(num_scenes_to_unroll)))

# Quantitative evaluation

In [ ]:
cle_evaluator.evaluate(sim_out)
validation_results = cle_evaluator.validation_results()
agg = ValidationCountingAggregator().aggregate(validation_results)
cle_evaluator.reset()
print(agg)

# Qualitative evaluation

In [ ]:
# TODO when we have visualisation